# Jan 2019 New functionalities examples

Example demonstrating how to use the torque_tube functionality.

#### Prerequisites (Step 0):
We assume all instalation is working at this point, reffer to the other journals for this! 

# 2-UP, Hexagonal Torque Tube Fixed Tilt Example

In [1]:
import os
testfolder = os.path.abspath(r'..\bifacial_radiance\TEMP')  

# You can alternatively point to an empty directory (it will open a load GUI Visual Interface)
# or specify any other directory in your computer. I.E.:
# testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo'

print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP


In [2]:
try:
    import bifacial_radiance
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.

import numpy as np

### Specify all variables for the module and scene

Below find a list of all of the possible parameters for makeModule. 
scene and simulation parameters are also organized below. 
This simulation will be a complete simulation in terms of parameters that you can modify.

In [9]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp

timestamp = 4020 # Noon, June 17th. 
simulationname = 'Torque_tube_hex_test'

## SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = 2.35    
azimuth_ang=90
lat = 37.5
lon = -77.6
nMods = 4   # doing a smaller array for better visualization on this example.
nRows = 2  

# MakeModule Parameters
module_type='my_custom_panel'
x = 1.996      # landscape, sinze x > y. Remember that orientation has been deprecated.
y = 0.991
tilt = 10
numpanels = 2  # doing a 2-up system!
cellLevelModule = False # not doing a cell level module on this example.

# Gaps:
xgap = 0.05  # distance between modules in the row.
ygap = 0.15  # distance between the 2 modules along the collector slope.
zgap = 0.175 # if there is a torquetube, this is the distance between the torquetube and the modules.
# If there is not a module, zgap is the distance between the module and the axis of rotation (relevant for 
# tracking systems. 

# TorqueTube Parameters
torqueTube = True
tubetype = 'Hex'
diameter = 0.15
torqueTubeMaterial = 'Metal_Grey'       # IT's NOT GRAY, IT's GREY.

### Run the example

In [10]:
demo = bifacial_radiance.RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(lat,lon) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit(metdata,timestamp)  # Noon, June 17th

# Making module with all the variables
moduledict=demo.makeModule(name=module_type,x=x,y=y,bifi=1, 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, rewriteModulefile = True, xgap = xgap)
# create a scene with all the variables. 
# Specifying the pitch automatically with the collector width (sceney) returend by moduledict.
# Height has been deprecated as an input. pass clearance_height or hub_height in the scenedict.
sceneDict = {'tilt':tilt,'pitch': np.round(moduledict['sceney'] / gcr,3),'clearance_height':clearance_height,'azimuth':azimuth_ang, 'module_type':module_type}  

scene = demo.makeScene(moduletype=module_type, sceneDict=sceneDict) #makeScene creates a .rad file of the Scene
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!


c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\main.py:2414: pvlibDeprecationWarning: The get_sun_rise_set_transit function was deprecated in pvlib 0.6.1 and will be removed in 0.7. Use sun_rise_set_transit_spa instead.
  sunup= pvlib.irradiance.solarposition.get_sun_rise_set_transit(datetimetz, lat, lon) #only for pvlib <0.6.1



Module Name: my_custom_panel
REWRITING pre-existing module file. 
Module my_custom_panel successfully created
Created Torque_tube_hex_test.oct


At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:
    
#### rvu -vf views\front.vp -e .01 Torque_tube_hex_test.oct
   
And then proceed happily with your analysis:

In [ ]:
analysis = bifacial_radiance.AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
analysis.analysis(octfile, demo.name, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )


And then you dance-dance-dance. 

Except that, at the current stage of the software, if you open the results you will notice that now some of the sensors are measuring the irradinace at the torque tube and must be removed. You can use read
resultsDF = read1Result(os.path.join('results',selectfile)) #return dataframe
cleanResult()

# Single-axis-tracking by HOUR example

Think of this as a fixed-tilt where the clearance height and the tracker tilt is changing. So the angles have to be re-calculated for every timestamp.

In [ ]:
#Some variables again. Assuming bifacial_radiance was imported.

timeindex = 4020 # Noon, June 17th. 
simulationname = '1axisTrack'+str(timeindex)

# MakeModule Parameters
module_type='my_little_pony_panel'
x = 1.996  # 2-up portrait Longi with 15cm additional gap
y = 0.991
tilt = 10

# Tracking Angle Calculation Parameters
hub_height = 2.35   
axis_azimuth = 180.0
roundTrackerAngleBool = True 
angledelta = 5
limit_angle = 45
backtrack = True
    
# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo

In [ ]:
#separate folder for this example
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo2'

In [ ]:
# Running the simulation
demo2 = bifacial_radiance.RadianceObj(simulationname, path = testfolder)  # Create a RadianceObj 'object' named 'demo'
demo2.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo2.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo2.readEPW(epwfile) # read in the weather data
demo2.gendaylit(metdata, timeindex)  # Noon, June 17th

moduledict=demo2.makeModule(name=module_type,x=x,y=y)    
tracker_theta, tracker_height, azimuth_ang = demo2.getTrackingGeometryTimeIndex(metdata, timeindex=timeindex, angledelta = angledelta, roundTrackerAngleBool = roundTrackerAngleBool, axis_azimuth = axis_azimuth, limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, hubheight = hub_height, sceney =  moduledict['sceney'] )
sceneDict = {'tilt':tracker_theta,'pitch': moduledict['sceney'] / gcr,'height':tracker_height,'azimuth':azimuth_ang}  
scene = demo2.makeScene(module_type, sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo2.makeOct(demo2.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.

At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:
    
   ## rvu -vf views\front.vp -e .01 Torque_tube_hex_test.oct
   
   And then proceed happily with your analysis:

In [ ]:
analysis = bifacial_radiance.AnalysisObj(octfile, demo2.basename)  # return an analysis object including the scan dimensions for back irradiance
frontDict, backDict = analysis.analysis(octfile, demo2.basename, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio: %0.3f - %0.3f' %(min(analysis.backRatio), np.mean(analysis.backRatio)) )

After you set up your system parameters, you can just change the timestamp and evaluate any day. You could also use gendaylit2manual to pass specific DNI/DHI if you so wanted.

Enjoy!

# Add Custom Elements to your Scene
This shows how to add a custom element, in this case a Cube, that will be placed in the center of your scene to mark the 0,0 location. A tracking-example with torque tube is used.

In [ ]:
#First all the Variables:
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\demo'

#Some variables again. Assuming bifacial_radiance was imported.

timeindex = 4020 # Noon, June 17th. 
simulationname = '1axisTrack'+str(timeindex)

# MakeModule Parameters
module_type='my_little_pony_panel'
x = 1.996  # 2-up portrait Longi with 15cm additional gap
y = 0.991
tilt = 10

# Tracking Angle Calculation Parameters
hub_height = 0.5   
axis_azimuth = 180.0
roundTrackerAngleBool = True 
angledelta = 5
limit_angle = 45
backtrack = True
    
# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo

# Running the simulation
demo2 = bifacial_radiance.RadianceObj(simulationname, path = testfolder)  # Create a RadianceObj 'object' named 'demo'
demo2.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo2.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo2.readEPW(epwfile) # read in the weather data
demo2.gendaylit(metdata, timeindex)  # Noon, June 17th

We are going to create a "Mycube.rad" file in the objects folder, right after we make the Module. 
This is a prism (so we use 'genbox'), that is black from the ground.rad list of materials ('black')
We are naming it 'cuteBox'
Its sides are going to be 0.5x0.5x0.5 m 
and We are going to leave its bottom surface coincident with the plane z=0, but going to center on X and Y.
The command for this description is :
>'! genbox black PVmodule 0.5 0.5 0.5 | xform -t -0.25 -0.25 0'


In [ ]:
name='Mycube'
text='! genbox black cuteBox 0.5 0.5 0.5 | xform -t -0.25 -0.25 0'
customObject = demo2.makeCustomObject(name,text)


We are saving the filename and address into customObject so we can call it when making the Scene and add it to it.

At this point, you can go to the objects folder and check that the object was created properly.
### objects\\Mycube.rad

Let's create teh SCene now:

In [ ]:
tracker_theta, tracker_height, azimuth_ang = demo2.getTrackingGeometryTimeIndex(metdata, timeindex=timeindex, angledelta = angledelta, roundTrackerAngleBool = roundTrackerAngleBool, axis_azimuth = axis_azimuth, limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, hubheight = hub_height, sceney = moduledict['sceney'] )
sceneDict = {'tilt':tracker_theta,'pitch': moduledict['sceney'] / gcr,'height':tracker_height,'azimuth':azimuth_ang}  
scene = demo2.makeScene(module_type, sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.

Now let's add the customObject to the Scene. We are not going to translate it or anything because we want it at the center, but you can pass translation, rotation, and any other XFORM command from Radiance.

I am passing a rotation 0 because xform has to have something (I think) otherwise it gets confused.

In [ ]:
demo2.appendtoScene(scene.radfile, customObject, '!xform -rz 0')

octfile = demo2.makeOct(demo2.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.

At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:
    
   #### rvu -vf views\front.vp -e .01 Torque_tube_hex_test.oct
   
And then proceed happily with your analysis. If any of the sensors hits the Box object we just created, the list of materials should say something with "cuteBox" on it. (That's why I set the clearance so low for this example, so the module and the cuteBox intercept).

In [ ]:
analysis = bifacial_radiance.AnalysisObj(octfile, demo2.basename)  # return an analysis object including the scan dimensions for back irradiance
frontDict, backDict = analysis.analysis(octfile, demo2.basename, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio: %0.3f - %0.3f' %(min(analysis.backRatio), np.mean(analysis.backRatio)) )

# Read and clean RESULTS!
When using a gap between modules (or a torque tube for 1-up), some of the materials might end up being "sky", "ground", torquetube, or even inner sides of the modules themselves if the resolution is very high.

The following is an example on how to use the read1Result and cleanResult functions of this last update.

In [ ]:
resultFile='C:\Users\sayala\Documents\RadianceScenes\Demo2\\results\irr_Torque_tube_hex_test.csv'
test = bifacial_radiance.load.read1Result(resultFile)
nmodules = 2 # 2-up configuration. 
sensorsy = 200 # this was the original resolution. Since we are removing some sensors 
               # that end up in the gap, the routine will interpolate to get this same number of sensors.
               # it will interpolate to 100 sensors per panel for the 2-up configuration for this example.
sazm = 270 # for the timestamp analyzed, the panels were facing West. This is important because the sensors and modules must be switched so it matches when the geometry is rotated...
Front4240, Back4240 = bifacial_radiance.load.deepcleanResult(test, sensorsy, nmodules, sazm, automatic=True)

In [ ]:
print("Read Results has {} material at position 100, with Front irradiance {}  W/m^2".format(test['mattype'][100],test['Wm2Front'][100]))
print("Cleaning the results removes this material, and leaves only modules.")
print("Now the Front Irradiance value is {} W/m^2".format(Front4240[100]))

import matplotlib.pyplot as plt
plt.plot(test['Wm2Front'], label='Results')
plt.plot(Front4240, label='Clean Results')
plt.legend()

## Reading Results and Cleaning 

When there is a torque tube, or a gap between panels, the irradiance result files in the result folders somethimes have values from the sky, the ground, the torque tube, or even the inner side of the panels.
The values you want are usually something like "...a0.PVModule.6457" and "...a0.PVModule.2310" for the front and back. If it is two panels, you might also want a1.

The following function can load one result file, and either ask the user to identify the material of interest (useful for debuging), or automatically find the correct surfaces and delete datapoints from the other surfaces.

Also, it then resamples to the original number of sensors (or a new one if desired) so that the results can be compared with other sets of results (particularly whenn tracking, the number of clean datapoints might vary slightly).

### IMPORTANT: 
AND!!!!!!!!!!!!!!!!!!!! It rotates the values of the sensors based on the panel facing East or West, so that the same "sensor" will get the right value. 

In [ ]:
ResultFile='C:\Users\sayala\Documents\RadianceScenes\\asdf2\\results\irr_PRISM_2UP_4234_WITHTorqueTube_Round_black_Diam_0.1_tubeZgap_0.15_GAP_0.05.csv'

#This reads the file
test = bifacial_radiance.load.read1Result(ResultFile)
numberofsensors=200
numberofpanels = 2
azimuth_angle = 270
automatic = True

#This cleans it.
Front4243, Back4243 = bifacial_radiance.load.deepcleanResult(test, numberofsensors, numberofpanels, azimuth_angle, automatic)

import matplotlib.pyplot as plt
plt.plot(Back4243)
plt.ylabel('Back Irradiance W/m2')
plt.xlabel('Position on the two panels (gap not plotted), panel 1 0 to 100, panel 2 101 to 200')
plt.show()


In [ ]:
cleanResult(test, 200, 2, 270)